In [2]:
## USER 테이블 build
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = (
    SparkSession.builder \
                .master("local")
                .appName("World Count")
                .getOrCreate()
)
file_path = r"C:\Users\brian\Desktop\JUNSOO\Project\ETL_DATA\data\ranking_2024-10-11.json"
df = spark.read.format('json').option("multiLine", True).load(file_path)

# 직업명을 제대로 채우기
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType , DoubleType
df = df.select(F.explode("ranking").alias("USER"))
df = df.select("USER.character_name",
               "USER.date",
               "USER.class_name",
               "USER.sub_class_name",
               "USER.character_level",
               "USER.character_exp",
               "USER.ranking")
df = df.withColumn("class",df["sub_class_name"])
df = df.withColumn("class",F.when(df["sub_class_name"]== "", df["class_name"]) \
                            .otherwise(df["class"]))
df = df.drop("class_name","sub_class_name")


# 각 유저의 위치한 지역 컬럼추가
df = df.withColumn("status",
                   F.when(df["character_level"]>=290,"Tallahart") \
                    .when((df["character_level"]<=289)&(df["character_level"]>=285),"Carcion") \
                    .when((df["character_level"]<=284)&(df["character_level"]>=280),"Arteria") \
                    .when((df["character_level"]<=279)&(df["character_level"]>=275),"Dowonkyung") \
                    .when((df["character_level"]<=274)&(df["character_level"]>=270),"Odium") \
                    .when((df["character_level"]<=269)&(df["character_level"]>=265),"HotelArcs") \
                    .when((df["character_level"]<=264)&(df["character_level"]>=260),"Cernium") \
                    .otherwise("AcaneRiver"))

file_path = r"C:\Users\brian\Desktop\JUNSOO\rankingflow\data\maple_exp.csv"
schema = StructType([
    StructField("row_number",IntegerType(),True),
    StructField("level",IntegerType(),True),
    StructField("need_exp",DoubleType(), True)
])
exp_df = spark.read.format('csv').schema(schema).option("multiLine", True).load(f"{file_path}")
exp_df.show(10)
exp_df = exp_df.dropna()
exp_df = exp_df.select("level","need_exp")
exp_df.show(10)


+----------+-----+-----------------+
|row_number|level|         need_exp|
+----------+-----+-----------------+
|      NULL| NULL|             NULL|
|         0|  260|1.731919984062E12|
|         1|  261|1.749239183902E12|
|         2|  262|1.766731575741E12|
|         3|  263|1.784398891498E12|
|         4|  264|1.802242880412E12|
|         5|  265|2.342915744535E12|
|         6|  266| 2.36634490198E12|
|         7|  267|2.390008350999E12|
|         8|  268|2.413908434508E12|
+----------+-----+-----------------+
only showing top 10 rows

+-----+-----------------+
|level|         need_exp|
+-----+-----------------+
|  260|1.731919984062E12|
|  261|1.749239183902E12|
|  262|1.766731575741E12|
|  263|1.784398891498E12|
|  264|1.802242880412E12|
|  265|2.342915744535E12|
|  266| 2.36634490198E12|
|  267|2.390008350999E12|
|  268|2.413908434508E12|
|  269|2.438047518853E12|
+-----+-----------------+
only showing top 10 rows



In [22]:
class_status = df.groupBy("class","date").pivot("status").agg(F.count("status"))
class_status.show()

+-----------------+----------+-------+-------+----------+---------+
|            class|      date|Arteria|Carcion|Dowonkyung|Tallahart|
+-----------------+----------+-------+-------+----------+---------+
|           카이저|2024-10-11|    146|     54|        85|       12|
|         블래스터|2024-10-11|     86|     36|        41|        5|
|             아란|2024-10-11|    159|    102|       104|       19|
|           초보자|2024-10-11|      5|   NULL|         6|     NULL|
|       패스파인더|2024-10-11|    410|    243|       278|       24|
|아크메이지(불,독)|2024-10-11|    549|    382|       338|       67|
|           시티즌|2024-10-11|      1|   NULL|         2|     NULL|
|             제논|2024-10-11|     92|     63|        74|       15|
|             라라|2024-10-11|    268|     87|       190|       16|
|       캐논마스터|2024-10-11|    156|    108|       115|       25|
|             제로|2024-10-11|    286|    151|       197|       30|
|             호영|2024-10-11|    304|    163|       185|       25|
|       보우마스터|2024-1